In [ ]:
# Checking where's currenly used python interpreter
!which python
!which pip

In [ ]:
# Installing requirements for WANN
!pip install -r requirements.txt

In [ ]:
# Run code belo to download mnist data from kaggle (maker sure, that you've kaggle module installed)

# !kaggle competitions download -c digit-recognizer
# !mkdir data
# !unzip digit-recognizer.zip -d ./data

In [ ]:
# Imports
from wann import WANN
from utils import WANNClassificationTrainer
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from random import choice
from matplotlib import pyplot as plt
import tensorflow as tf

Let's take a look at our data

In [ ]:
# Loading data
train_df = pd.read_csv('../data/train.csv')
y_train = train_df.pop('label')
test_df = pd.read_csv('../data/test.csv')

In [ ]:
# Show raw dataframe
train_df

In [ ]:
# Conversion: pandas dataframe to numpy array
X_train = np.array(train_df).astype('float') / 255
y_train = y_train.values
X_test = np.array(test_df).astype('float') / 255
columns = list(train_df.columns)
del train_df
del test_df

In [ ]:
# Show some images
fig, ((ax, ay), (az, aw)) = plt.subplots(2, 2)

ax.imshow(choice(X_train).reshape((28, 28)))
ay.imshow(choice(X_train).reshape((28, 28)))
az.imshow(choice(X_train).reshape((28, 28)))
aw.imshow(choice(X_train).reshape((28, 28)))

plt.show()

Performing actual training

In [ ]:
trainer = WANNClassificationTrainer(sensor_nodes=columns, output_nodes=[str(i) for i in range(10)])
wanns = trainer.fit(X_train, y_train)